In [50]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
!pip install overrides

In [52]:
!pip install allennlp

In [0]:
from pathlib import Path
from typing import *

import torch
import torch.optim as optim

import numpy as np
import pandas as pd
from functools import partial
from overrides import overrides

## В AllenNLP каждый обучающий пример должен быть представлен как объект типа Instance, содержащий различные Fields.
from allennlp.data import Instance
## Существует множество различных способов представить слово в виде индексов. 
## Например, можно создать словарь уникальных слов, где каждое слово имеет свой id. 
## Или же можно проиндексировать каждый уникальный символ, тогда слово будет представлено последовательностью этих индексов.
## AllenNLP использует абстракцию TokenIndexer для этого представления.
from allennlp.data.token_indexers import TokenIndexer
from allennlp.data.tokenizers import Token
from allennlp.nn import util as nn_util

In [0]:
class Config(dict):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        for k, v in kwargs.items():
            setattr(self, k, v)
    
    def set(self, key, val):
        self[key] = val
        setattr(self, key, val)
        
config = Config(
    testing=True,
    seed=1,
    batch_size=64,
    lr=3e-4,
    epochs=2,
    hidden_sz=64,
    max_seq_len=100, # necessary to limit memory usage
    max_vocab_size=100000,
)

In [0]:
from allennlp.common.checks import ConfigurationError

In [0]:
USE_GPU = torch.cuda.is_available()

Set random seed manually to replicate results

In [57]:
torch.manual_seed(config.seed)

В большинстве случаев пайплайн AllenNLP состоит из следующих элементов:

**DatasetReader**: Извлекает необходимую информацию из входных данных и представляет ее в виде последовательности объектов типа Instance.

**Model**: Модель, которая будет обучаться.

**Iterator**: Делит данные на батчи.

**Trainer**: Определяет процесс обучения модели.

(**Predictor**: Генерирует предсказания для сырых текстовых примеров.)

# Загрузка данных

В большинстве случаев для решения задач с помощью AllenNLP, нужно реализовать два класса. Первый - **DatasetReader**, который заключает всю логику чтения данных из файла и их преобразования в список Instances.

In [0]:
from allennlp.data.dataset_readers import DatasetReader

**TokenIndexer** содержит правила преобразования токенов в индексы, 
Словарь (**Vocabulary**) содержит соответствующие отображения строк в целые числа.
Например, индексатов для токена может указывать, что токен должен быть прдставлен как последовательность составляющих его индексов, в этом случае **Vocabulary** будет содержать отображения вида *{character -> id}*.


In [0]:
from allennlp.data.vocabulary import Vocabulary

### Подготовка dataset-а

In [0]:
label_cols = ["toxic", "severe_toxic", "obscene",
              "threat", "insult", "identity_hate"]

**DatasetReader** одна из важнейших частей пайплайна. 

**DatasetReader** включает следующие пункты:

1.   чтение данных;
2.   выделение важной информации из данных;
3.   преобразование данных в список объектов типа Instances.

В AllenNLP нет класса Dataset. DatasetReaders представляет собой схему для преобразования сырых данных в список Instances.

Field отвечают за преобразование данных в тензоры, можно определить свой класс Field.

В AllenNLP уже определено несколько типов полей, один из самых важных - **TextField**.

**TextField** преобразует последовательность токенов в целые числа. 

**НО!** он не производит препроцессинг, не токенизирует предложение и т.д., это нужно делать самостоятельно.

Есть дополнительный аргумент **token indexer**.

In [0]:
from allennlp.data.fields import TextField, MetadataField, ArrayField

In [0]:
class JigsawDatasetReader(DatasetReader):
  ## Параметр dict с TokenIndexer, который определяет, как конвертировать токены в индексы. 
    def __init__(self, tokenizer: Callable[[str], List[str]]=lambda x: x.split(),
                 token_indexers: Dict[str, TokenIndexer] = None,
                 max_seq_len: Optional[int]=config.max_seq_len) -> None:
        super().__init__(lazy=False)
        self.tokenizer = tokenizer
        self.token_indexers = token_indexers or {"tokens": SingleIdTokenIndexer()}
        self.max_seq_len = max_seq_len

    ##  Обрабатывает сам текст, метки классов, метаданные и т.д. 
    @overrides
    def text_to_instance(self, tokens: List[Token], id: str=None,
                         labels: np.ndarray=None) -> Instance:
        sentence_field = TextField(tokens, self.token_indexers)
        fields = {"tokens": sentence_field}
        
        id_field = MetadataField(id)
        fields["id"] = id_field
        
        if labels is None:
            labels = np.zeros(len(label_cols))
        label_field = ArrayField(array=labels)
        fields["label"] = label_field

        return Instance(fields)

    ## _read получает имя файла, считывает данные, преобразует их в список объекто типа Instances.
    @overrides
    def _read(self, file_path: str) -> Iterator[Instance]:
        df = pd.read_csv(file_path)
        if config.testing: df = df.head(1000)
        for i, row in df.iterrows():
            yield self.text_to_instance(
                [Token(x) for x in self.tokenizer(row["comment_text"])],
                row["id"], row[label_cols].values,
            )

Итак, **DatasetReader** считывает входные данные и преобразует их в список *Instances*. *Instances* состоят из *Fields*, которые определяют сами данные и способ их обработки.

In [64]:
! wget https://www.dropbox.com/s/ezddhu2okrxmfop/test_proced.csv
! wget https://www.dropbox.com/s/fwilrzm2vhs48m2/test.csv
! wget https://www.dropbox.com/s/51dbf7zg9m4q61c/train.csv
! wget https://www.dropbox.com/s/1qi10pqo1afh0i0/valid.csv

--2019-10-29 20:54:47--  https://www.dropbox.com/s/ezddhu2okrxmfop/test_proced.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:601f:1::a27d:901
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/ezddhu2okrxmfop/test_proced.csv [following]
--2019-10-29 20:54:48--  https://www.dropbox.com/s/raw/ezddhu2okrxmfop/test_proced.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc59578ab722dc9cf6d38bf74c07.dl.dropboxusercontent.com/cd/0/inline/ArVAOl4Rs-RsRX7_m0GuV4Txl7JGgxcLROBGesnfiyJWzAlFcmfdcxHsaTzB3I1fifpgkVhcEfbdRghuo0EiS1Q0KmQr685ulXdnBYmdaQkWQSxRJrRZR4LyKaXsZTZm52M/file# [following]
--2019-10-29 20:54:48--  https://uc59578ab722dc9cf6d38bf74c07.dl.dropboxusercontent.com/cd/0/inline/ArVAOl4Rs-RsRX7_m0GuV4Txl7JGgxcLROBGesnfiyJWzAlFcmfdcxHsaTzB3I1fifpgkVhcEfbdRghuo0EiS1Q0KmQr685ulXdnBYmda

### Подготовка обработчиков токенов

**PretrainedBertIndexer** - TokenIndexer, соответствующий предобученной модели Bert.

In [0]:
from allennlp.data.token_indexers import PretrainedBertIndexer

token_indexer = PretrainedBertIndexer(
    pretrained_model="bert-base-uncased",
    max_pieces=config.max_seq_len,
    do_lowercase=True,
 )

def tokenizer(s: str):
    return token_indexer.wordpiece_tokenizer(s)[:config.max_seq_len - 2]

In [0]:
reader = JigsawDatasetReader(
    tokenizer=tokenizer,
    token_indexers={"tokens": token_indexer}
)

In [67]:
train_ds, test_ds = (reader.read(fname) for fname in ["train.csv", "test_proced.csv"])
val_ds = None


0it [00:00, ?it/s]
77it [00:00, 766.10it/s]
175it [00:00, 818.78it/s]
267it [00:00, 892.98it/s]
0it [00:00, ?it/s]
88it [00:00, 873.83it/s]
176it [00:00, 875.37it/s]
251it [00:00, 881.20it/s]

In [68]:
len(train_ds)

267

In [69]:
train_ds[:10]

In [70]:
vars(train_ds[0].fields["tokens"])

{'_indexed_tokens': None,
 '_indexer_name_to_indexed_token': None,
 '_token_index_to_indexer_name': None,
 '_token_indexers': {'tokens': <allennlp.data.token_indexers.wordpiece_indexer.PretrainedBertIndexer at 0x7f0f7d78e8d0>},
 'tokens': [[UNK],
  [UNK],
  the,
  edit,
  ##s,
  made,
  under,
  my,
  user,
  ##name,
  [UNK],
  [UNK],
  [UNK],
  were,
  reverted,
  ##?,
  [UNK],
  weren,
  ##',
  ##t,
  van,
  ##dal,
  ##isms,
  ##,,
  just,
  closure,
  on,
  some,
  [UNK],
  after,
  [UNK],
  voted,
  at,
  [UNK],
  [UNK],
  [UNK],
  [UNK],
  [UNK],
  please,
  don,
  ##',
  ##t,
  remove,
  the,
  template,
  from,
  the,
  talk,
  page,
  since,
  [UNK],
  retired,
  now,
  ##.,
  ##8,
  ##9,
  ##.,
  ##20,
  ##5,
  ##.,
  ##38,
  ##.,
  ##27]}

Предполагалось, что здесь будут тензоры, но, чтобы конвертировать поля (fields) в тензоры, нужно создать словарь (vocabulary) для наших данных.

### Подготовка словаря (vocabulary)

В нашем случае не нужно строить словарь Bert indexer сам задает нужно отображение токенов в индексы.

In [0]:
vocab = Vocabulary()

### Подготовка итератора

Итератор отвечает за деление данных на батчи и их подготовку для модели.

В AllenNLP определно несколько удобных итераторов. Будем использовать **BucketIterator**. Он группирует предложения приблизительно одинаковой длины для минимизации паддинга.

In [0]:
from allennlp.data.iterators import BucketIterator

In [0]:
iterator = BucketIterator(batch_size=config.batch_size, 
                          sorting_keys=[("tokens", "num_tokens")],
                         )

Обязательно нужно напомнить итератору, что все Instances проиндексированы в нашем словаре (vocabulary), т.е., что все строки были преобразованы в целые числа, используя заданное ранее отображение.

In [0]:
iterator.index_with(vocab)

### Подготовленные данные

In [0]:
batch = next(iter(iterator(train_ds)))

In [88]:
batch

{'id': ['00a54f5b8d29a2b7',
  '0037fe4f8f5cdcfb',
  '0023daf96917e0d0',
  '000897889268bc93',
  '008faa76dd3eb890',
  '005de39c51ef844a',
  '009a3333aa4ac011',
  '00a00eb46f78b1d2',
  '00190820581d90ce',
  '00a656a81c6f0903',
  '0002bcb3da6cb337',
  '00908f946bd2fd05',
  '006a4e493b506d1e',
  '001ffdcc3e7fb49c',
  '00a3073791a5feaa',
  '004d912a00f79c89',
  '0057e30091cf3e81',
  '005fb1983ff191e9',
  '0036e50f42d0b679',
  '009ce66bcd5de288',
  '006eaaaca322e12d',
  '008fdd0ca51cdadc',
  '004af8a71399a4dc',
  '009d0fe5e7ee720a',
  '0007e25b2121310b',
  '0086998b34865f93',
  '006b888560bcdfcd',
  '00128363e367d703',
  '004b97c80705a548',
  '006a263a08b593c5',
  '00040093b2687caa',
  '00025465d4725e87',
  '000c6a3f0cd3ba8e',
  '006e87872c8b370c',
  '00a9d33dea92c45c',
  '00472b8e2d38d1ea',
  '009cb63b1bd5daf9',
  '000bfd0867774845',
  '002b90cc8a94c76b',
  '0083ead5c8afc356',
  '0091798f05a311af',
  '003bd094feef5263',
  '00582dcd527c8d7d',
  '0083790fbfe5014d',
  '001325b8b20ea8aa',
  '0

In [89]:
batch["tokens"]["tokens"]

tensor([[  101,   100,  2005,  ...,     0,     0,     0],
        [  101,   100,  3752,  ...,     0,     0,     0],
        [  101,   100,   100,  ...,     0,     0,     0],
        ...,
        [  101,   100,  2064,  ...,     0,     0,     0],
        [  101,   100,   100,  ...,  2592, 29632,   102],
        [  101,   100,   100,  ...,   100,   102,     0]])

In [90]:
batch["tokens"]["tokens"].shape

torch.Size([64, 24])

# Подготовка модели

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim

Помимо класса **DatasetReader**, обычно нужно реализовать класс **Model**, который является модулем PyTorch. На вход подается тензор, на выходе словарь выходных тензоров (включающий training loss, которую нужно оптимизировать).

AllenNLP модели обычно состоят из:

token embedder;

encoder;

(Для seq-to-seq моделей) decoder.

In [0]:
from allennlp.models import Model

from allennlp.modules.seq2vec_encoders import Seq2VecEncoder, PytorchSeq2VecWrapper
from allennlp.nn.util import get_text_field_mask
from allennlp.modules.text_field_embedders import TextFieldEmbedder

class BaselineModel(Model):
    def __init__(self, word_embeddings: TextFieldEmbedder,
                 encoder: Seq2VecEncoder,
                 out_sz: int=len(label_cols)):
      ## Конструктору класса нужно передать словарь (vocab)
        super().__init__(vocab)
        self.word_embeddings = word_embeddings
        self.encoder = encoder
        self.projection = nn.Linear(self.encoder.get_output_dim(), out_sz)
        self.loss = nn.BCEWithLogitsLoss()
        
    def forward(self, tokens: Dict[str, torch.Tensor],
                id: Any, label: torch.Tensor) -> torch.Tensor:
        ## Из-за паддинга нужно вводить маску, чтобы модель видела только информативные части предложений. 
        mask = get_text_field_mask(tokens)
        embeddings = self.word_embeddings(tokens)
        state = self.encoder(embeddings, mask)
        class_logits = self.projection(state)
        
        output = {"class_logits": class_logits}
        output["loss"] = self.loss(class_logits, label)

        return output

### Готовим эмбеддинги

In [0]:
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder
from allennlp.modules.token_embedders.bert_token_embedder import PretrainedBertEmbedder

bert_embedder = PretrainedBertEmbedder(
        pretrained_model="bert-base-uncased",
        top_layer_only=True, # conserve memory
)
word_embeddings: TextFieldEmbedder = BasicTextFieldEmbedder({"tokens": bert_embedder},
                                                            # Чтобы игнорировать замаскированные занчения
                                                           allow_unmatched_keys = True)

In [0]:
BERT_DIM = word_embeddings.get_output_dim()

class BertSentencePooler(Seq2VecEncoder):
    def forward(self, embs: torch.tensor, 
                mask: torch.tensor=None) -> torch.tensor:

        return embs[:, 0]
    
    @overrides
    def get_output_dim(self) -> int:
        return BERT_DIM
    
encoder = BertSentencePooler(vocab)

Строим модель

In [0]:
model = BaselineModel(
    word_embeddings, 
    encoder, 
)

In [0]:
if USE_GPU: model.cuda()
else: model

# Проверим модель

In [0]:
batch = nn_util.move_to_device(batch, 0 if USE_GPU else -1)

In [0]:
tokens = batch["tokens"]
labels = batch

In [99]:
tokens

{'mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 0]]),
 'tokens': tensor([[  101,   100,  2005,  ...,     0,     0,     0],
         [  101,   100,  3752,  ...,     0,     0,     0],
         [  101,   100,   100,  ...,     0,     0,     0],
         ...,
         [  101,   100,  2064,  ...,     0,     0,     0],
         [  101,   100,   100,  ...,  2592, 29632,   102],
         [  101,   100,   100,  ...,   100,   102,     0]]),
 'tokens-offsets': tensor([[ 1,  2,  3,  ...,  0,  0,  0],
         [ 1,  2,  3,  ...,  0,  0,  0],
         [ 1,  2,  3,  ...,  0,  0,  0],
         ...,
         [ 1,  2,  3,  ...,  0,  0,  0],
         [ 1,  2,  3,  ..., 20, 21, 22],
         [ 1,  2,  3,  ..., 20, 21,  0]]),
 'tokens-type-ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0

In [100]:
mask = get_text_field_mask(tokens)
mask

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 0]])

In [101]:
embeddings = model.word_embeddings(tokens)
state = model.encoder(embeddings, mask)
class_logits = model.projection(state)
class_logits

tensor([[-0.1145,  0.0215, -0.5080, -0.1537, -0.1508,  0.2334],
        [ 0.1976, -0.0708, -0.5108, -0.2801,  0.0134,  0.0271],
        [ 0.0355, -0.2590, -0.5887, -0.2327,  0.3112,  0.0500],
        [ 0.0603, -0.1349, -0.4211, -0.2369,  0.2386,  0.1047],
        [ 0.2617,  0.3407, -0.2248, -0.0359, -0.0774, -0.2127],
        [ 0.2123,  0.0323, -0.3265, -0.4256,  0.0512,  0.2372],
        [-0.0228,  0.0486, -0.3513, -0.1816, -0.0951,  0.1991],
        [ 0.5909,  0.4791,  0.1234,  0.2549, -0.3621, -0.2005],
        [-0.0272, -0.0012, -0.5064, -0.4777,  0.1700, -0.0691],
        [-0.0246, -0.1369, -0.5717, -0.1650, -0.0271,  0.2038],
        [ 0.0760,  0.1233, -0.4525, -0.3145,  0.2415,  0.0473],
        [-0.1123, -0.1083, -0.2333, -0.4683,  0.1232,  0.1124],
        [ 0.0139,  0.0444, -0.3812, -0.4663,  0.1955, -0.2172],
        [-0.0663, -0.1316, -0.5453, -0.3915,  0.0501,  0.1232],
        [-0.0659, -0.2313, -0.4383, -0.4111,  0.1205, -0.0384],
        [ 0.0496, -0.0408, -0.4771, -0.3

In [102]:
model(**batch)

{'class_logits': tensor([[ 0.0688,  0.1159, -0.4146, -0.0672, -0.3661,  0.1553],
         [ 0.1483,  0.0911, -0.3735, -0.5372,  0.0036, -0.0632],
         [ 0.0740, -0.0963, -0.4739, -0.0403,  0.3503,  0.0397],
         [ 0.0645, -0.0680, -0.5565, -0.2188,  0.2389,  0.0317],
         [ 0.7737,  0.4068, -0.0085,  0.2519, -0.2855, -0.2457],
         [ 0.0905, -0.0771, -0.3815, -0.2523,  0.1467,  0.0729],
         [-0.1899,  0.2512, -0.1674, -0.2583, -0.0818,  0.1700],
         [ 0.4894,  0.4504, -0.0246, -0.2127, -0.1714,  0.0332],
         [ 0.1698, -0.0262, -0.4746, -0.0137, -0.0602,  0.0724],
         [-0.1229, -0.1556, -0.4624, -0.4269,  0.0276,  0.0286],
         [-0.0507, -0.1426, -0.2402, -0.2189,  0.3800,  0.0253],
         [-0.0325, -0.0888, -0.2879, -0.4762,  0.1077,  0.0559],
         [-0.1679, -0.1422, -0.4375, -0.4351,  0.2096, -0.0198],
         [ 0.0596, -0.0994, -0.4359, -0.3350,  0.1306,  0.0740],
         [-0.0465, -0.0735, -0.3893, -0.4971,  0.1423, -0.0697],
         

In [0]:
loss = model(**batch)["loss"]

In [104]:
loss

tensor(0.6578, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)

In [0]:
loss.backward()

# Обучение

In [0]:
optimizer = optim.Adam(model.parameters(), lr=config.lr)

In [0]:
## Используем класс Trainer из AllenNLP
from allennlp.training.trainer import Trainer

trainer = Trainer(
    model=model,
    optimizer=optimizer,
    iterator=iterator,
    train_dataset=train_ds,
    cuda_device=0 if USE_GPU else -1,
    num_epochs=config.epochs,
)

In [110]:
metrics = trainer.train()

unable to check gpu_memory_mb(), continuing
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/allennlp/common/util.py", line 378, in gpu_memory_mb
    encoding='utf-8')
  File "/usr/lib/python3.6/subprocess.py", line 356, in check_output
    **kwargs).stdout
  File "/usr/lib/python3.6/subprocess.py", line 438, in run
    output=stdout, stderr=stderr)
subprocess.CalledProcessError: Command '['nvidia-smi', '--query-gpu=memory.used', '--format=csv,nounits,noheader']' returned non-zero exit status 9.

  0%|          | 0/5 [00:00<?, ?it/s]
loss: 0.6710 ||:  20%|██        | 1/5 [00:21<01:24, 21.24s/it]
loss: 0.6574 ||:  40%|████      | 2/5 [00:26<00:49, 16.38s/it]
loss: 0.6492 ||:  60%|██████    | 3/5 [00:30<00:25, 12.66s/it]
loss: 0.6411 ||:  80%|████████  | 4/5 [00:52<00:15, 15.59s/it]
loss: 0.6324 ||: 100%|██████████| 5/5 [01:03<00:00, 14.02s/it]
unable to check gpu_memory_mb(), continuing
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/

# Генерация предиктора

AllenNLP содержит абстракцию **Predictor**, которая получает на вход данные, конвертирует их в Instances, скармливает модели и возвращает результат. 

В AllenNLP есть только один реализованный предиктор **SentenceTaggerPredictor**. Для большинства задач предиктор нужно реализовать самостоятельно.

In [0]:
from allennlp.data.iterators import DataIterator
from tqdm import tqdm
from scipy.special import expit # the sigmoid function

def tonp(tsr): return tsr.detach().cpu().numpy()

class Predictor:
    def __init__(self, model: Model, iterator: DataIterator,
                 cuda_device: int=-1) -> None:
        self.model = model
        self.iterator = iterator
        self.cuda_device = cuda_device
        
    def _extract_data(self, batch) -> np.ndarray:
        out_dict = self.model(**batch)
        return expit(tonp(out_dict["class_logits"]))
    
    def predict(self, ds: Iterable[Instance]) -> np.ndarray:
        pred_generator = self.iterator(ds, num_epochs=1, shuffle=False)
        self.model.eval()
        pred_generator_tqdm = tqdm(pred_generator,
                                   total=self.iterator.get_num_batches(ds))
        preds = []
        with torch.no_grad():
            for batch in pred_generator_tqdm:
                batch = nn_util.move_to_device(batch, self.cuda_device)
                preds.append(self._extract_data(batch))
        return np.concatenate(preds, axis=0)

In [0]:
from allennlp.data.iterators import BasicIterator
# проходим по всему набору данных, не меняя порядка
seq_iterator = BasicIterator(batch_size=64)
seq_iterator.index_with(vocab)

In [113]:
predictor = Predictor(model, seq_iterator, cuda_device=0 if USE_GPU else -1)
train_preds = predictor.predict(train_ds) 
test_preds = predictor.predict(test_ds) 


100%|██████████| 5/5 [01:28<00:00, 15.94s/it]

100%|██████████| 4/4 [01:23<00:00, 20.69s/it]


In [127]:
test_preds


array([[0.4129021 , 0.38936403, 0.28252792, 0.35432905, 0.40151322,
        0.4339282 ],
       [0.3979461 , 0.3819608 , 0.34652203, 0.31746748, 0.4500172 ,
        0.40407592],
       [0.42484435, 0.3842783 , 0.3517712 , 0.3583979 , 0.3973712 ,
        0.43223068],
       ...,
       [0.39026165, 0.379195  , 0.31677172, 0.34768498, 0.40115   ,
        0.41660818],
       [0.40983585, 0.36331385, 0.35998017, 0.33345345, 0.37998417,
        0.43453267],
       [0.41559297, 0.4271993 , 0.34390524, 0.42004392, 0.3018501 ,
        0.46570572]], dtype=float32)

In [0]:
check_test = pd.read_csv("test_proced.csv")

In [131]:
check_test.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0001ea8717f6de06,Thank you for understanding. I think very high...,0,0,0,0,0,0
1,000247e83dcc1211,:Dear god this site is horrible.,0,0,0,0,0,0
2,0002f87b16116a7f,"""::: Somebody will invariably try to add Relig...",0,0,0,0,0,0
3,0003e1cccfd5a40a,""" \n\n It says it right there that it IS a typ...",0,0,0,0,0,0
4,00059ace3e3e9a53,""" \n\n == Before adding a new product to the l...",0,0,0,0,0,0
